In [ ]:
# Step 1: Install Required Libraries
!pip install shap lime

In [ ]:
# Step 2: Load the Fine-Tuned Model and Tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("fine_tuned_model")


In [ ]:
# Step 3: Prepare a Sample Input
sample_text = "ለኮንዶሚኒየም ለጠባብ ቤቶች ገላግሌ የሆነ ከንፁህ የሲልከን ጥሬ እቃ የተሰራ የልጆች ማጠቢያ ምስሉ ላይ እንደሚያዩት መታጠፍ መዘርጋት የሚችል 3350ብር ይደውሉልን"


In [ ]:
# Step 4: Interpret Predictions with SHAP
import shap

# Define a prediction function for SHAP
def predict_ner(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, is_split_into_words=False)
    outputs = model(**inputs).logits
    return outputs.detach().numpy()

# Initialize SHAP explainer
explainer = shap.Explainer(predict_ner, tokenizer)

# Compute SHAP values for the sample text
shap_values = explainer([sample_text])

# Visualize SHAP values
print("SHAP Visualization:")
shap.plots.text(shap_values)

In [ ]:
# Step 5: Interpret Predictions with LIME
from lime.lime_text import LimeTextExplainer

# Initialize LIME explainer
explainer_lime = LimeTextExplainer(class_names=list(id2label.values()))

# Define a prediction function for LIME
def predict_proba(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, is_split_into_words=False)
    outputs = model(**inputs).logits
    return outputs.softmax(dim=-1).detach().numpy()

# Explain the sample text
exp = explainer_lime.explain_instance(sample_text, predict_proba, num_features=10, top_labels=3)

# Display the LIME explanation
print("LIME Visualization:")
exp.show_in_notebook()

In [ ]:
# Step 6: Analyze Difficult Cases
difficult_text = "በ1000 ብር የሚሸጥ ስልክ በገለገለ ውስጥ ይገኛል"

# Apply SHAP to the difficult case
shap_values_difficult = explainer([difficult_text])
print("SHAP Visualization for Difficult Case:")
shap.plots.text(shap_values_difficult)

# Apply LIME to the difficult case
exp_difficult = explainer_lime.explain_instance(difficult_text, predict_proba, num_features=10, top_labels=3)
print("LIME Visualization for Difficult Case:")
exp_difficult.show_in_notebook()



In [ ]:
# Step 7: Save Interpretability Results
# Save SHAP visualization
shap.plots.text(shap_values, show=False).savefig("shap_visualization.png")

# Save LIME visualization
exp.save_to_file("lime_visualization.html")

print("Interpretability results saved successfully!")